# JAX Performance Benchmark - Jupyter Kernel Execution

This notebook tests JAX performance when executed through a Jupyter kernel.
Compare results with direct script and jupyter-book execution.

In [ ]:
import time
import platform
import os

print("=" * 60)
print("JUPYTER KERNEL EXECUTION BENCHMARK")
print("=" * 60)
print(f"Platform: {platform.platform()}")
print(f"Python: {platform.python_version()}")
print(f"CPU Count: {os.cpu_count()}")

In [ ]:
# Import JAX and check devices
import jax
import jax.numpy as jnp

devices = jax.devices()
default_backend = jax.default_backend()
has_gpu = any('cuda' in str(d).lower() or 'gpu' in str(d).lower() for d in devices)

print(f"JAX devices: {devices}")
print(f"Default backend: {default_backend}")
print(f"GPU Available: {has_gpu}")

In [ ]:
# Define JIT-compiled function
@jax.jit
def matmul(a, b):
    return jnp.dot(a, b)

print("matmul function defined with @jax.jit")

In [ ]:
# Benchmark 1: Small matrix (1000x1000) - includes JIT compilation
print("\n" + "=" * 60)
print("BENCHMARK 1: Small Matrix (1000x1000)")
print("=" * 60)

n = 1000
key = jax.random.PRNGKey(0)
A = jax.random.normal(key, (n, n))
B = jax.random.normal(key, (n, n))

# Warm-up run (includes compilation)
start = time.perf_counter()
C = matmul(A, B).block_until_ready()
warmup_time = time.perf_counter() - start
print(f"Warm-up (includes JIT compile): {warmup_time:.3f} seconds")

# Compiled run
start = time.perf_counter()
C = matmul(A, B).block_until_ready()
compiled_time = time.perf_counter() - start
print(f"Compiled execution: {compiled_time:.3f} seconds")

In [ ]:
# Benchmark 2: Large matrix (3000x3000) - triggers recompilation
print("\n" + "=" * 60)
print("BENCHMARK 2: Large Matrix (3000x3000)")
print("=" * 60)

n = 3000
A = jax.random.normal(key, (n, n))
B = jax.random.normal(key, (n, n))

# Warm-up run (recompilation for new size)
start = time.perf_counter()
C = matmul(A, B).block_until_ready()
warmup_time = time.perf_counter() - start
print(f"Warm-up (recompile for new size): {warmup_time:.3f} seconds")

# Compiled run
start = time.perf_counter()
C = matmul(A, B).block_until_ready()
compiled_time = time.perf_counter() - start
print(f"Compiled execution: {compiled_time:.3f} seconds")

In [ ]:
# Benchmark 3: Element-wise operations (50M elements)
print("\n" + "=" * 60)
print("BENCHMARK 3: Element-wise Operations (50M elements)")
print("=" * 60)

@jax.jit
def elementwise_ops(x):
    return jnp.cos(x**2) + jnp.sin(x)

x = jax.random.normal(key, (50_000_000,))

# Warm-up
start = time.perf_counter()
y = elementwise_ops(x).block_until_ready()
warmup_time = time.perf_counter() - start
print(f"Warm-up (includes JIT compile): {warmup_time:.3f} seconds")

# Compiled
start = time.perf_counter()
y = elementwise_ops(x).block_until_ready()
compiled_time = time.perf_counter() - start
print(f"Compiled execution: {compiled_time:.3f} seconds")

In [ ]:
# Benchmark 4: Multiple small operations (simulates lecture cells)
print("\n" + "=" * 60)
print("BENCHMARK 4: Multiple Small Operations (lecture simulation)")
print("=" * 60)

total_start = time.perf_counter()

# Simulate multiple cell executions with different operations
for i, size in enumerate([100, 500, 1000, 2000, 3000]):
    @jax.jit
    def compute(a, b):
        return jnp.dot(a, b) + jnp.sum(a)
    
    A = jax.random.normal(key, (size, size))
    B = jax.random.normal(key, (size, size))
    
    start = time.perf_counter()
    result = compute(A, B).block_until_ready()
    elapsed = time.perf_counter() - start
    print(f"  Size {size}x{size}: {elapsed:.3f} seconds")

total_time = time.perf_counter() - total_start
print(f"\nTotal time for all operations: {total_time:.3f} seconds")

In [ ]:
print("\n" + "=" * 60)
print("JUPYTER KERNEL EXECUTION BENCHMARK COMPLETE")
print("=" * 60)